In [1]:
import torch
import matplotlib.pyplot as plt
from torch.optim import Adam
from utils.pipeline import forward, get_latent_mean_std, get_latent_from_text

/home/langrieger/miniconda3/envs/Text2Head/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
lat_mean, lat_std = get_latent_mean_std()
lat_rep = (torch.randn(lat_mean.shape) * lat_std * 0.85 + lat_mean).detach().requires_grad_(True)

In [ ]:
optimizer = Adam(params=[lat_rep],
                 lr=0.001, 
                 maximize=True)

In [ ]:
camera_params = {
        "camera_distance": 1.42,
        "camera_angle": 55.,
        "focal_length": 3.15,
        "max_ray_length": (0.5 + 1) * 3.15 + 1.5,
        # Image
        "resolution_y": 100,
        "resolution_x": 100
    }

phong_params = {
    "ambient_coeff": 0.67,
    "diffuse_coeff": 0.79,
    "specular_coeff": 0.3,  # 0.63,
    "shininess": 1.,
    # Colors
    "object_color": torch.tensor([0.61, 0.61, 0.61]),  # torch.tensor([0.63, 0.17, 0.78]),
    "background_color": torch.tensor([0., 0., 0.])  # torch.tensor([0.35, 0.94, 0.26])
}

light_params = {
    "amb_light_color": torch.tensor([0.57, 0.07, 0.69]),
    # light 1
    "light_intensity_1": 0.,  # 1.1,
    "light_color_1": torch.tensor([0.88, 0.99, 0.74]),
    "light_dir_1": torch.tensor([-0.41, -0.51, -0.76]),
    # light p
    "light_intensity_p": 1.,
    "light_color_p": torch.tensor([0.88, 0.99, 0.74]),
    "light_pos_p": torch.tensor([2., 0., 2.])
}

In [ ]:
score = forward(lat_rep, "untextured render of a face", camera_params, phong_params, light_params)

In [ ]:
score[0].backward()

In [2]:
latent, hist = get_latent_from_text("A woman", n_updates=3)

update step 0 - score: tensor([[22.5312]], device='cuda:0', dtype=torch.float16,
       grad_fn=<MmBackward0>)
lr:  0.003


OutOfMemoryError: CUDA out of memory. Tried to allocate 306.00 MiB (GPU 0; 10.91 GiB total capacity; 10.18 GiB already allocated; 53.44 MiB free; 10.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
plt.plot(torch.tensor(hist["scores"]).numpy())

In [ ]:
torch.save(latent, "optim_latent")

In [ ]:
torch.save(torch.stack(hist["latents"]), "latent_history")

In [ ]:
torch.save(torch.stack(hist["images"]), "render_history")

In [ ]:
plt.imshow(hist["images"][3].detach().numpy())
plt.axis('off')  # Turn off axes
plt.show()
#plt.savefig(f"optim_img/high_lr_{i}", bbox_inches='tight')

In [ ]:
for i, im in enumerate(hist["images"]):
    plt.imshow(im.detach().numpy())
    plt.axis('off')  # Turn off axes
    #plt.show()
    plt.savefig(f"optim_img/high_lr_{5+i}", bbox_inches='tight')